In [45]:
#import libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam

import random
np.random.seed(32)
random.seed(32)

In [59]:
# Load the MNIST Fashion dataset
filename = 'fashion-mnist.csv'
df = pd.read_csv(filename)

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# Normalize pixel values between [0, 1]
X = X / 255

# Only use 50% of data
rand = random.sample(range(10000), 5000)
X = X[rand]
y = y[rand]

# One-hot encoding
y = to_categorical(y)

# Split the dataset into training and test data sets
X_left, X_gen, y_left, y_gen = train_test_split(X, y, test_size=0.2, random_state=420)
X_train, X_test, y_train, y_test = train_test_split(X_left, y_left, test_size=0.25, random_state=420)

In [60]:
# Define the CNN model
def create_model(learning_rate, dropout_rate):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate = learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [69]:
# Define the search space for the hyperparameters
search_space = [Real(1e-4, 1e-2, name='learning_rate'),
                Real(0.0, 0.5, name='dropout_rate')]

In [70]:
# Define the objective function to minimize (i.e., 1 - accuracy)
acc_list = []

@use_named_args(search_space)
def objective(**params):
    # Create the model
    model = create_model(**params)

    # Fit the model to the training data
    model.fit(X_train.reshape(-1, 28, 28, 1), y_train, epochs=10, verbose=0)

    # Predict the labels for the test data
    y_pred = model.predict(X_test.reshape(-1, 28, 28, 1))

    # Convert the predicted labels to integers
    y_pred = np.argmax(y_pred, axis=1)

    # Compute the accuracy of the model
    accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)

    acc_list.append(accuracy)

    # Return 1 - accuracy (to minimize)
    return 1.0 - accuracy

In [54]:
# Perform Bayesian optimization
result = gp_minimize(objective, search_space, n_calls=10, random_state=420, acq_func="gp_hedge")

# Print the best set of hyperparameters and the corresponding accuracy
print(f'Best hyperparameters: {result.x}')
print(f'Best accuracy: {1.0 - result.fun}')

# Print prior test accuracies
print(acc_list)

32/32 [==============================] - 0s 3ms/step
Best hyperparameters: [0.005135041012503236, 0.1642231502145]
Best accuracy: 0.854
[]


In [58]:
# Define optimal model with found hyperparameters
opt_model = create_model(*result.x)

# Fit the model to the training data
opt_model.fit(X_train.reshape(-1, 28, 28, 1), y_train, epochs=10, verbose=0)

# Predict the labels for the test data
y_pred = opt_model.predict(X_gen.reshape(-1, 28, 28, 1))

# Convert the predicted labels to integers
y_pred = np.argmax(y_pred, axis=1)

# Compute the accuracy of the model
accuracy = accuracy_score(np.argmax(y_gen, axis=1), y_pred)

# Print accuracy for optimal model
print(accuracy)

32/32 [==============================] - 0s 5ms/step
0.845
[0.854, 0.816, 0.852, 0.829, 0.849, 0.837, 0.843, 0.836, 0.846, 0.822]


In [75]:
# Random sampling hyperparameters
rand_lr = [random.uniform(1e-4, 1e-2) for _ in range(20)]
rand_dr = [random.uniform(0.0, 0.5) for _ in range(20)]

for rand in zip(rand_lr, rand_dr):
    objective(rand)

best_idx = np.argmax(acc_list[10:19])
best_acc = acc_list[best_idx]
best_lr = rand_lr[best_idx - 10]
best_dr = rand_dr[best_idx - 10]
print(best_acc, best_lr, best_dr)

32/32 [==============================] - 1s 8ms/step


ValueError: attempt to get argmax of an empty sequence

In [80]:
# Statisical test and confidence interval


[0.859, 0.858, 0.848, 0.817, 0.864, 0.848, 0.84, 0.847, 0.865, 0.846]
0.865 0.001739088273958544 0.1193796360022038
